In [1]:
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_excel('hellofresh.xlsx',
                  sheet_name=['Context', 'Questions', 
                            '1-Order data','2-Product data',
                            '3-Customer data', '4-Prediction data'])
df_order = df['1-Order data']
df_order = df_order[['fk_customer', 'Delivery week', 'Region', 'Channel','fk_product', 'Product sku', 'Year','Week', 'Date','Comp Key']]
df_order.set_index('fk_customer', inplace=True)
df_order.head()

,Delivery week,Region,Channel,fk_product,Product sku,Year,Week,Date,Comp Key
fk_customer,,,,,,,,,
402956,2017-W01,bundaberg,Direct Sales Agency,830,AU-FMB-4-4-0,2017,1,2017-01-02,402956 02/01/17
404468,2017-W01,adelaide-2,Direct Sales Agency,1061,AU-CB-3-2-0,2017,1,2017-01-02,404468 02/01/17
404519,2017-W01,gold-coast,Direct Sales Agency,830,AU-FMB-4-4-0,2017,1,2017-01-02,404519 02/01/17
405966,2017-W01,wollongong-1,Direct Sales Agency,1061,AU-CB-3-2-0,2017,1,2017-01-02,405966 02/01/17
406404,2017-W01,shepparton,App - Activation Organic,1061,AU-CB-3-2-0,2017,1,2017-01-02,406404 02/01/17


## Filtering

I used pandas datetime function to turn the `Date` column into datetime objects. I then created two dataframes for two different situations. `six_weeks` is used to see the box orders from week 2-6 as it was stated that every customer should have an order in week 1. 


In [18]:
df_order.Date = pd.to_datetime(df_order.Date)
inc_6_weeks = dt.timedelta(weeks=5)
inc_52_weeks = dt.timedelta(weeks=52)
first_week = df_order[df_order['Date'] > dt.date(2017,1,2)] ##Skip the first week
first_6 = pd.Timestamp(2017,1,2) + inc_6_weeks 
first_52 = pd.Timestamp(2017,1,2) + inc_52_weeks
six_weeks = first_week[first_week['Date'] < first_6] ##Between 1st week (non-inclusive) and 6th week (inclusive)
ft_wk = df_order[df_order['Date'] < first_52] ## This is the dataframe for the first year (52 weeks)
ft_wk.head()


,Delivery week,Region,Channel,fk_product,Product sku,Year,Week,Date,Comp Key
fk_customer,,,,,,,,,
402956,2017-W01,bundaberg,Direct Sales Agency,830,AU-FMB-4-4-0,2017,1,2017-01-02,402956 02/01/17
404468,2017-W01,adelaide-2,Direct Sales Agency,1061,AU-CB-3-2-0,2017,1,2017-01-02,404468 02/01/17
404519,2017-W01,gold-coast,Direct Sales Agency,830,AU-FMB-4-4-0,2017,1,2017-01-02,404519 02/01/17
405966,2017-W01,wollongong-1,Direct Sales Agency,1061,AU-CB-3-2-0,2017,1,2017-01-02,405966 02/01/17
406404,2017-W01,shepparton,App - Activation Organic,1061,AU-CB-3-2-0,2017,1,2017-01-02,406404 02/01/17


## 7) This shows all the customers that ordered a box from week 2-6 and also the amount of boxes.


In [19]:
ft_orders = ft_wk.groupby(['fk_customer']).agg(['count'])
box_orders = six_weeks.groupby(['fk_customer']).agg(['count'])
box_orders['Delivery week']
box_orders['Ordered Box?'] = 'Yes'
count_orders = box_orders[['Delivery week', 'Ordered Box?']]
count_ft = ft_orders[['Delivery week']]
count_orders

,Delivery week,Ordered Box?
,count,
fk_customer,,
115583,1,Yes
186610,3,Yes
249176,3,Yes
258405,1,Yes
318637,2,Yes
321215,2,Yes
323176,2,Yes
330655,1,Yes


## 8) Total number of boxes each customer ordered in their first 52 weeks

In [17]:
count_ft

,Delivery week
,count
fk_customer,
10294,1
13823,2
28456,1
30261,1
34519,2
35778,1
35947,5
40980,8


## Note:

I do realise that the answers given in this notebook isn't exactly in the format needed, but I did not manage my time properly and it is very late.
I also was excited to do the machine learning part of this, but I have run out of time. If required I am happy to finish off question 9 if given a time extension!